<a href="https://colab.research.google.com/github/gcosma/COP509/blob/main/Tutorials/Tutorial7Summarization_with_user_pasted_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abstractive summarisation models

In [ ]:
# @title Step 1: Give the paragraph you want to summarise.
# concatenated_texts_list = A number of the messages are from people who are facing up to their own worries about cancer. Rebecca Stead from Macmillan Cancer Support told BBC News. Hearing the news that you have cancer is a huge moment in anyone's life and there is no right or wrong way to respond. Going through waves of different emotions is completely normal.We do know, however, that many people will experience uncertainty or worry.This could be about practical matters such as paying the mortgage or being confused by the treatment being given, she says, urging people to get advice.The King's openness about having cancer has also been praised as helping remove taboos around the disease. This could be about practical matters such as paying the mortgage or being confused by the treatment being given, she says, urging people to get advice. According to Macmillan's, the King's public acknowledgement of his cancer prompted a surge in people seeking information.
concatenated_texts_list = input("Paste your data: ")
# print(concatenated_texts_list)

Paste your data: A quintessential English nanny, Mary Poppins is a slightly stern but caring woman, who uses magic and self-control to take care of the Banks children.[1] She is usually identifiable by her sensible hat and parrot umbrella, which she brings with her wherever she goes on outings; she also has the power to teleport to a place she wants. She is kind towards the children, but can be firm when needed. She is "practically perfect in every way". In the film version, she is a young woman, with an air of grace and elegance about her.  Author P. L. Travers was very firm about Mary Poppins' appearance in the novel's illustrations, working closely with illustrator Mary Shepard to create an image of the character. Eventually they based Mary Poppins' appearance on that of a Dutch doll: tall and bony, with short black hair, large blue eyes, a snub nose, and a prim, pursed mouth. Travers originally objected to the casting of Julie Andrews as Mary Poppins, claiming that Andrews was too 

## Model 1: primera

In [ ]:
# @title Load model and summarise
# Importing necessary modules
from IPython.display import clear_output
from transformers import (
    AutoTokenizer,
    LEDForConditionalGeneration,
    LEDConfig,
)

import torch
# Initializing variables
TOKENIZER = AutoTokenizer.from_pretrained("allenai/PRIMERA-multinews")
CONFIG = LEDConfig.from_pretrained("allenai/PRIMERA-multinews")
MODEL = LEDForConditionalGeneration.from_pretrained("allenai/PRIMERA-multinews", config=CONFIG)
# MODEL.gradient_checkpointing_enable()
PAD_TOKEN_ID = TOKENIZER.pad_token_id
DOCSEP_TOKEN_ID = TOKENIZER.convert_tokens_to_ids("<doc-sep>")

# Use a pipeline as a high-level helper
import torch
from transformers import pipeline

pipe = pipeline(
    task = "text2text-generation",
    model = MODEL,
    tokenizer = TOKENIZER,
    torch_dtype=torch.bfloat16,
)

# Use model
result = pipe(
    concatenated_texts_list,
    use_cache = True,
    min_length = 128, # It is recommended to have it at least around 64
    num_beams = 5,
    max_length = 1024, # Has to be at least higher than min_length
    pad_token_id = TOKENIZER.pad_token_id,
    bos_token_id = TOKENIZER.bos_token_id,
    eos_token_id = TOKENIZER.eos_token_id,
    do_sample=True, # Only necessary to enable if want to use temperature or top_p parameters
    temperature=0.1, # Will control 'randomness' of the answer
    top_p=0.3
    )

clear_output()
print(result)

[{'generated_text': '– Mary Poppins was "practically perfect in every way" when she met Julie Andrews for the first time, according to P. L. Travers, who originally objected to Andrews as the character because she was too attractive. But upon meeting Andrews in person, Travers found her to be much more perfect than she had anticipated. She is "a quintessential English nanny, a slightly stern but caring woman, who uses magic and self-control to take care of the Banks children," Travers wrote in the novel. In the film version, she is a young woman, "with an air of grace and elegance about her." She is usually identifiable by her sensible hat and parrot umbrella, which she brings with her wherever she goes on outings; she also has the power to teleport to a place she wants.'}]


## Model 2: BRIO Model

In [ ]:
# @title Load model and summarise
from transformers import BartTokenizer, PegasusTokenizer
from transformers import BartForConditionalGeneration, PegasusForConditionalGeneration

IS_CNNDM = True # whether to use CNNDM dataset (BART-base) or XSum dataset (PEGASUS-base)
LOWER = False

# Load our model checkpoints
if IS_CNNDM:
    model = BartForConditionalGeneration.from_pretrained('Yale-LILY/brio-cnndm-uncased')
    tokenizer = BartTokenizer.from_pretrained('Yale-LILY/brio-cnndm-uncased')
else:
    model = PegasusForConditionalGeneration.from_pretrained('Yale-LILY/brio-xsum-cased')
    tokenizer = PegasusTokenizer.from_pretrained('Yale-LILY/brio-xsum-cased')

max_length = 1024 if IS_CNNDM else 512


# Tokenize the text
input_ids = tokenizer.encode(concatenated_texts_list, return_tensors='pt')

# Generate summary with the model
summary_ids = model.generate(input_ids, max_length=max_length)

# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

clear_output()
# Print the summary
print(summary)

Mary Poppins is a quintessential English nanny. She is a stern but caring woman who uses magic and self-control to care for the Banks children. P. L. Travers based her appearance on a Dutch doll. she is identifiable by her sensible hat and parrot umbrella.


## Model 4:  EFactsum

In [ ]:
# @title Load model and summarise
from transformers import BartTokenizer, PegasusTokenizer
from transformers import BartForConditionalGeneration, PegasusForConditionalGeneration

IS_CNNDM = True
max_length = 1024 if IS_CNNDM else 512

if IS_CNNDM:
    model = BartForConditionalGeneration.from_pretrained('tanay/efactsum-bart-cnndm')
    tokenizer = BartTokenizer.from_pretrained('tanay/efactsum-bart-cnndm')
else:
    model = PegasusForConditionalGeneration.from_pretrained('tanay/efactsum-pegasus-xsum')
    tokenizer = PegasusTokenizer.from_pretrained('tanay/efactsum-pegasus-xsum')

# Tokenize the text
input_ids = tokenizer.encode(concatenated_texts_list, return_tensors='pt')

# Generate summary with the model
summary_ids = model.generate(input_ids, max_length=max_length)

# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)


clear_output()
# Print the summary
print(summary)


a quintessential English nanny , Mary Poppins is a slightly stern but caring woman . she uses magic and self-control to take care of the Banks children . she is usually identifiable by her sensible hat and parrot umbrella . she also has the power to teleport to a place she wants .


## Model 3:  Unlimiformer model

In [ ]:
# @title load model and code

%cd /content/
!git clone https://github.com/abertsch72/unlimiformer.git
!pip install -q -r unlimiformer/requirements.txt
!pip install -q faiss-cpu
%cd unlimiformer/src

from unlimiformer import Unlimiformer
from random_training_unlimiformer import RandomTrainingUnlimiformer
from usage import UnlimiformerArguments, training_addin

from transformers import BartForConditionalGeneration, AutoTokenizer
from datasets import load_dataset
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# example using govreport
modelname = "abertsch/unlimiformer-bart-govreport-alternating"

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained(modelname)

defaults = UnlimiformerArguments()
unlimiformer_kwargs = {
            'layer_begin': defaults.layer_begin,
            'layer_end': defaults.layer_end,
            'unlimiformer_head_num': defaults.unlimiformer_head_num,
            'exclude_attention': defaults.unlimiformer_exclude,
            'chunk_overlap': defaults.unlimiformer_chunk_overlap,
            'model_encoder_max_len': defaults.unlimiformer_chunk_size,
            'verbose': defaults.unlimiformer_verbose, 'tokenizer': tokenizer,
            'unlimiformer_training': defaults.unlimiformer_training,
            'use_datastore': defaults.use_datastore,
            'flat_index': defaults.flat_index,
            'test_datastore': defaults.test_datastore,
            'reconstruct_embeddings': defaults.reconstruct_embeddings,
            'gpu_datastore': defaults.gpu_datastore,
            'gpu_index': defaults.gpu_index
}

model.to(device)

model = Unlimiformer.convert_model(model, **unlimiformer_kwargs)
model.eval()
model.to(device)

# Tokenize the text
input_ids = tokenizer.encode(concatenated_texts_list, return_tensors='pt')

# Generate summary with the model
summary_ids = model.generate(input_ids, max_length=max_length)

# Decode the summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

clear_output()
# Print the summary
print(summary)

 Mary Poppins is a quintessential English nanny, about whom the Banks children depend. This report provides a brief description of Mary Poppingins' role as an English Nanny; see CRS Report RS22533, Mary Po'ins: A Sketch of her Work, by [author name scrubbed].


# Extractive summarization methods

## Model 1:  BERTSUM

In [ ]:
# @title load the model and summarise
%cd /content/
!pip install -q bert-extractive-summarizer
from summarizer import Summarizer
model = Summarizer()
# Summarize the text
summary = model(concatenated_texts_list, num_sentences=3)

clear_output()
# Print the summary
print("- " + summary + "\n")

- A quintessential English nanny, Mary Poppins is a slightly stern but caring woman, who uses magic and self-control to take care of the Banks children.[1] She is usually identifiable by her sensible hat and parrot umbrella, which she brings with her wherever she goes on outings; she also has the power to teleport to a place she wants. She is kind towards the children, but can be firm when needed. Author P. L. Travers was very firm about Mary Poppins' appearance in the novel's illustrations, working closely with illustrator Mary Shepard to create an image of the character.



## Model 2:  Sbert

In [ ]:
# @title load the model and summarise
%cd /content/
!pip install -U -q sentence-transformers
from summarizer.sbert import SBertSummarizer

model = SBertSummarizer('paraphrase-MiniLM-L6-v2')

# Summarize the text
summary = model(concatenated_texts_list, num_sentences=3)

clear_output()
# Print the summary
print("- " + summary + "\n")

- A quintessential English nanny, Mary Poppins is a slightly stern but caring woman, who uses magic and self-control to take care of the Banks children.[1] She is usually identifiable by her sensible hat and parrot umbrella, which she brings with her wherever she goes on outings; she also has the power to teleport to a place she wants. In the film version, she is a young woman, with an air of grace and elegance about her. Author P. L. Travers was very firm about Mary Poppins' appearance in the novel's illustrations, working closely with illustrator Mary Shepard to create an image of the character.

